In [ ]:
# !pip install mediapipe
# !git config --global http.sslVerify false
# !pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
import glob
origin_data = glob.glob(r"archive/Videos/*")
print(len(origin_data))

In [ ]:
import tensorflow as tf

In [ ]:
origin_data_violent = glob.glob(r"archive/Videos/Violent_*")
origin_data_normal = glob.glob(r"archive/Videos/Normal_*")
print(len(origin_data_violent))
print(len(origin_data_normal))

In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense,Dropout, GlobalAveragePooling2D, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
import math
import mediapipe as mp
from mediapipe.python.solutions import pose as mp_pose
import torch
from threading import Thread
import pandas as pd
import numpy as np
from threading import Thread

In [ ]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5n')
mp_drawing = mp.solutions.drawing_utils
mp_pose =mp.solutions.pose

Lấy keypoint 1 video

In [ ]:
# index_global = 0
def EachVideo(linkVideo): # This function is used to cycle through a video
    X = []
    video_path =r""+ linkVideo  # The path to the data file I use
    print(f"playing : {video_path}")
    cap = cv2.VideoCapture(video_path)
    skipTime  = 0
    skipFrame = 0
    while True:    
        ret, frame = cap.read()
        skipTime = skipTime +1
        if not ret:
            break
        if 1==1:
        # if skipTime >= 30: # When skipTime has passed the first 30 frames, ie the first 1 second, proceed to Detect Person
            skipFrame = skipFrame +1 # The variable skipFrame means that every 5 frames I will detect 1 time, so in 1 second I will detect 6 times
            # print(skipFrame)
            # if  skipFrame  == 5: # When skipFrame = 5, I will detect person and assign skipFrame = 0 to run again
            if 1==1:
                skipFrame = 0
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame.flags.writeable = False  
                result = yolo_model(frame)     # Detect Person
                frame.flags.writeable = True   
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                for (xmin, ymin, xmax,   ymax,  confidence,  clas) in result.xyxy[0].tolist(): # Loop through all the Persons present in the video, giving the x,y of each Person
                    c_lm = []
                    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
                            
                            # frame.flags.writeable = False  

                            resulta = pose.process(frame[int(ymin):int(ymax),int(xmin):int(xmax):])
                            # frame.flags.writeable = True   

                            if resulta.pose_landmarks and clas==0: # class here is class, class = 0 means human
                                for (id, lm) in enumerate(resulta.pose_landmarks.landmark):
                                    if id > 10 and id not in [17,18,19,20,21,22] and id not in [29,30,31,32] :
                                        c_lm.append(lm.x)
                                        c_lm.append(lm.y)
                                        # c_lm.append(lm.z)
                                        # c_lm.append(lm.visibility)
                    if len(c_lm) > 0: # c_ lm used to save a person's x and y variables in a loop through each person, when saving, there will be a state that there is no x,y data to save 
                        X.append(c_lm) # with linkVideo being violent, we add data to X_violent
                # if cv2.waitKey(1) & 0xFF == ord('q'):
                #       break
    cap.release()
    cv2.destroyAllWindows()
    return X


### Loop through multiple videos to get keypoint

In [ ]:
def AllVideo(startt,endd,dataset): # This function is used to cycle through all videos
    # Status (linkVideo) is : violent, non-violent
    X = []
    
    # cam (camIndex) is : cam1 or cam2
    # number of videos is : from startt to endd, for example non-violent dataset is startt 1->61 with cam 1, violent from 1 to 115 with cam 1
    for id,i in enumerate(range(startt,endd)):
        print(f'{startt} -> {endd} ## index: {startt+id}')
        X.extend(EachVideo(dataset[i])) # Implement the function to detect
    return X

### Save keypoint, run getData function to run multiple videos at the same time (threading) to get keypoint

In [ ]:
X = []
def getData(startt,endd,origin_data):
    global X
    X = AllVideo(startt,endd,origin_data)

    
active = []
skipT = 1500
for i in range(0,10):
    active_1 = Thread(target=getData,args=(skipT,skipT+20,origin_data_normal))
    skipT = skipT + 20
    if skipT >= 1600:
        break
    active.append(active_1)

for i in active:
    i.start()
for i in active:
    i.join()
    
print(len(X))
X = np.array(X)
print(X.shape)



# X = []
# def getData(startt,endd,origin_data):
#     global X
#     X = AllVideo(startt,endd,origin_data)

    
# active = []
# skipT = 0
# for i in range(0,10):
#     active_1 = Thread(target=getData,args=(skipT,skipT+20,origin_data_violent))
#     skipT = skipT + 20
#     active.append(active_1)

# for i in active:
#     i.start()
# for i in active:
#     i.join()
    
# print(len(X))
# X = np.array(X)
# print(X.shape)

In [ ]:
print(len(X))
Xt = np.array(X)
print(Xt.shape)

### Save keypoint to file

In [ ]:
Xt = pd.DataFrame(Xt)
Xt  = np.nan_to_num(Xt) # Khi đã có dữ liệu, sẽ có một số phần từ trong dữ liệu bị NaN, nên chuyển thành 0
Xt = pd.DataFrame(Xt)
Xt.to_csv("./normal_test.csv")

### Data Crane

Model 1

In [ ]:
violent_1_200 = pd.read_csv("voilent_0_650_final.csv")
# violent_200_400 = pd.read_csv("../input/subkevin/violent_200_400.csv")
# violent_400_600 = pd.read_csv("../input/subkevin/violent_400_600.csv")

normal_1_500 = pd.read_csv("normal_2000_2500_final.csv")
normal_500_1000 = pd.read_csv("normal_test.csv")
# normal_1000_1500 = pd.read_csv("../input/subkevin/normal_1000_1500.csv")
# normal_1500_2000 = pd.read_csv("../input/subkevin/normal_1500_2000.csv")
# normal_2000_2500 = pd.read_csv("normal_2000_2500_final.csv")

# violent_1_200 = violent_1_200.append(violent_200_400)
# violent_1_200 = violent_1_200.append(violent_400_600)


normal_1_500 = normal_1_500.append(normal_500_1000)
# normal_1_500 = normal_1_500.append(normal_1000_1500)
# normal_1_500 = normal_1_500.append(normal_1500_2000)
# normal_1_500 = normal_1_500.append(normal_2000_2500)


X = []
y = []

X_violent = violent_1_200.iloc[:,1:].values
len_X_violent = len(X_violent)
for i in range(10,len_X_violent):
    X.append(X_violent[i-10:i,:])
    y.append(1)
    
    
X_non_violent = normal_1_500.iloc[:,1:].values
len_X_non_violent = len(X_non_violent)
for i in range(10,len_X_non_violent):
    X.append(X_non_violent[i-10:i,:])
    y.append(0)
    
    
X = np.array(X)
y = np.array(y)    
print(violent_1_200.shape)
print(normal_1_500.shape)
print(X.shape)
print(y.shape)



In [ ]:
X_violent.shape

In [ ]:
# np.save('../siameseNet/data/normal_testX.npy', X)
# np.save('../siameseNet/data/normal_testY.npy', y)

In [ ]:
X.shape[1], X.shape[2]

In [ ]:
y = y.reshape(y.shape[0], -1)

In [ ]:
model  = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1],X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(Dense(units = 12, activation="relu"))
model.add(GlobalAveragePooling1D())
model.add(Dense(units = 1, activation="sigmoid"))
model.compile(optimizer="SGD",  metrics = ['accuracy'], loss = "binary_crossentropy")

In [ ]:
model.summary()

In [ ]:
model.input

In [ ]:
model.fit(X, y, epochs=32, batch_size=32, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the training history to see whether you're overfitting.
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['TRAIN', 'VAL'], loc='lower right')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import itertools


X_test = X.copy()
y_test = y.copy()
class_names_test = [0, 1]
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """Plots the confusion matrix."""
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                  horizontalalignment="center",
                  color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Classify pose in the TEST dataset using the trained model
y_pred = model.predict(X_test)

y_pred = [1 if i > 0.5 else 0 for i in y_pred]

# Convert the prediction result to class name
# y_pred_label = [class_names_test[i] for i in np.argmax(y_pred, axis=1)]
# y_true_label = list(y_test) #[class_names_test[i] for i in y_test]

# Plot the confusion matrix
cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm,
                      class_names_test,
                      title ='Confusion Matrix of Pose Classification Model')

# Print the classification report
print('\nClassification Report:\n', classification_report(list(y_test), y_pred))

In [ ]:
model.save("models/tf_k_first.h5")

In [ ]:
model.save("models/pb_model/", save_format=tf, save_traces=False)

In [ ]:
X_test[1]

Model 2

In [ ]:
def getAngle(array_a):
    len_a = len(array_a)
    len_x  = int(len_a/2)
    so_chan = 0
    fa = []
    for i in range(0,len_x):
        fa.append(so_chan)
        so_chan = so_chan + 2
        
    ff = []
    dem = 0
    sum  = 0
    for i in range(4,len_a):
        ff.append([])
        for j in range(i-4,i+1):
            
            len_x  = int(len(array_a[j])/2)
            for i in range(0,len_x):
                fa.append(so_chan)
                so_chan = so_chan + 2

            for k in range(0,len_x-1):
                for m in range(k+1,len_x):
                    tru_x =  array_a[j][fa[m]] - array_a[j][fa[k]] 
                    tru_y = array_a[j][fa[m]+1] -  array_a[j][fa[k]+1]
                    if tru_x == 0:
                        tru_x=1
                    atan = math.atan(tru_y / tru_x)
                    ff[dem].append(atan)
                    
        dem = dem + 1
    return ff

In [ ]:
normal_1_500_copy = np.copy(normal_1_500)
violent_1_200_copy = np.copy(violent_1_200)
                             
normal_1_500_array = list(normal_1_500_copy)
violent_1_200_array = list(violent_1_200_copy)

X_model_2_violent  = []
X_model_2_violent.extend(getAngle(violent_1_200_array))
X_model_2_violent = pd.DataFrame(X_model_2_violent)


X_model_2_non_violent = []
X_model_2_non_violent.extend(getAngle(normal_1_500_array))
X_model_2_non_violent = pd.DataFrame(X_model_2_non_violent)


X_2 = list([])
y_2 = list([])
# print(len(X_model_2_violent))

X_model_2_violent =  X_model_2_violent.iloc[:,1:].values
len_X_model_2_violent = len(X_model_2_violent)
for i in range(10,len_X_model_2_violent):
    X_2.append(X_model_2_violent[i-10:i,:])
    y_2.append(1)
    

X_model_2_non_violent =  X_model_2_non_violent.iloc[:,1:].values
len_X_model_2_non_violent = len(X_model_2_non_violent)
for i in range(10,len_X_model_2_non_violent):
    X_2.append(X_model_2_non_violent[i-10:i,:])
    y_2.append(0)


# y_model_2 = []

# len_X_violent = len(violent_1_200_array)
# for i in range(10,len_X_violent):
#     X_model_2.append(X_violent[i-10:i,:])
#     y.append(1)
# X = []
# y = []
X_2 = np.array(X_2)
y_2 = np.array(y_2)
y_2 = y_2.reshape(y_2.shape[0], -1)

print(X_2.shape)
print(y_2.shape)

In [ ]:
model_3  = Sequential()

model_3.add(Dense( 500, activation="relu",input_shape = (X_2.shape[1],X_2.shape[2])))
model_3.add(Dense( 300, activation="relu"))

# model_3.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1],X.shape[2])))
model_3.add(Dropout(0.4))

model_3.add(Dense( 100, activation="relu"))
model_3.add(Dense( 50, activation="relu"))
model_3.add(Dropout(0.3))

model_3.add(Dense( 25, activation="relu"))
model_3.add(Dense( 10, activation="relu"))
model_3.add(Dense( 5, activation="relu"))
model_3.add(Dense( 1, activation="sigmoid"))
model_3.compile(optimizer="adam",  metrics = ['accuracy'], loss = "binary_crossentropy")
model_3.summary()

In [ ]:
model_3.fit(X_2, y_2, epochs=32, batch_size=32, validation_split=0.2)

Data artlab

In [ ]:
X= []
y =[]
non_violent = pd.read_csv("normal_2000_2500_final.csv")
X_non_violent = non_violent.iloc[:,1:].values
len_X_non_violent = len(X_non_violent)
for i in range(10,len_X_non_violent):
  X.append(X_non_violent[i-10:i,:])
  y.append(0)
violent = pd.read_csv("voilent_0_650_final.csv")
X_violent = violent.iloc[:,1:].values
len_X_violent = len(X_violent)
for i in range(10,len_X_violent):
  X.append(X_violent[i-10:i,:])
  y.append(1)


In [ ]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

In [ ]:
model2  = Sequential()
model2.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1],X.shape[2])))
model2.add(Dropout(0.2))
model2.add(LSTM(units = 50, return_sequences = True))
model2.add(Dropout(0.2))
model2.add(LSTM(units = 50, return_sequences = True))
model2.add(Dropout(0.2))
model2.add(Dense(units = 4, activation="relu"))
model2.add(Dense(units = 1, activation="sigmoid"))
model2.compile(optimizer="adam",  metrics = ['accuracy'], loss = "binary_crossentropy")

In [ ]:
model2.fit(X, y, epochs=32, batch_size=32,validation_split=0.2)

In [ ]:
model.save("./model_v41.h5")